In [15]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer

from sklearn.manifold import TSNE
import plotly.express as px


In [16]:
'''
Would need OPENAI Keys in a file called azure-configuration.json with params:
    key
    endpoint
    deployment_name
'''

#Checking if keys are working properly

with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [17]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')   #Loading IAB Labels

In [18]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [19]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [20]:
def clean(item):
    item=item.replace("_", " ")
    item=item.lower()
    return item

In [21]:
df_MAF['item'].apply(clean)

0               crane
1          disk brake
2        auto factory
3                 cab
4          car mirror
            ...      
1872          'stole'
1873     'stone wall'
1874    cutting board
1875     electric ray
1876       infant cry
Name: item, Length: 1877, dtype: object

In [22]:
maf=df_MAF['item'].to_list()

In [23]:
call=iab_labels+maf

In [24]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [25]:
total_embeddings=[]

In [26]:
#Creating embeddings. Good to create it once and store it


def create_embeddings(item,total_embeddings):    
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

In [27]:
for items in tqdm(call):
    try:
        items=items.replace("&", 'and')
    except:
        pass

#only for Chat GPT version
    # if items in scores.keys():
    #     # print('here')
    #     key = list(scores[items].keys())[0]
    #     create_embeddings(key,total_embeddings)
    #     continue

    create_embeddings(items,total_embeddings)

100%|██████████| 2037/2037 [04:48<00:00,  7.05it/s]


In [28]:
len(total_embeddings)

2037

In [29]:
iab_embeddings=total_embeddings[0:160]   #Change this if you want to test a new external keyword set

In [30]:
maf_embeddings=total_embeddings[160:]

In [31]:
nodes=maf

In [32]:
# with open("data/problematic.pkl", 'rb') as f:
#     problematic=pkl.load(f)

In [33]:
# with open("data/child_parent_dic.pkl", 'rb') as f:
#     child_parent_dic=pkl.load(f)

In [34]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_labels=[]
top_n_cosines=[]
tsne_embeddings=[]



for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)

    new_similarities=[x for x in similarities if x[2]>=0.765]   #LOW THRESHOLD TO SEE GENERAL CLUSTER VALUES

    if len(new_similarities)==0:
        new_similarities.append(['No_Pred','No_Pred',[[0]]])
    
    temp_iab_labels=[]
    temp_cosines=[]


    for items in new_similarities:
        tsne_embeddings.append(maf_embeddings[i])

        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0])
        top_n_nodes.append(nodes[i])



    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)



100%|██████████| 1877/1877 [04:05<00:00,  7.64it/s]


In [35]:
len(top_n_labels_new) #THIS IS THE NUMBER OF MAPPINGS. KEEPING IT HIGH HERE TO HAVE MORE SAMPLES FOR CLUSTERING

122984

In [36]:
dic={'item':top_n_nodes, 
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [37]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [38]:
df_MAF_new.to_csv('results/OpenAI_MAF_INTERMEDIATE_0.785_TSNE_Mapping.csv')

In [39]:
df_tiers=pd.read_csv('data/MAFlabel_to_Adword_mappings_202210_combined_mappings.csv')

In [40]:
df_tiers=df_tiers['adword']

In [41]:
tier1=[]
all_tiers=df_tiers.to_list()
for tier in all_tiers:
    try:
        tier1.append(tier.split('|')[0])
    except:
        tier1.append(tier)

tier1=list(set(tier1))

In [42]:
len(tier1)

26

In [43]:
dic={'embeddings':tsne_embeddings, 'labels':top_n_labels_new, 'item': top_n_nodes}
df_labels_embeddings=pd.DataFrame.from_dict(dic)

In [44]:
df_labels_embeddings.head

<bound method NDFrame.head of                                                embeddings             labels  \
0       [-0.016414720565080643, -0.027218835428357124,...              Birds   
1       [-0.016414720565080643, -0.027218835428357124,...  Commercial Trucks   
2       [-0.016414720565080643, -0.027218835428357124,...            Cruises   
3       [-0.016414720565080643, -0.027218835428357124,...      Pickup Trucks   
4       [-0.016414720565080643, -0.027218835428357124,...        Convertible   
...                                                   ...                ...   
122979  [-0.011204354465007782, 0.002425432438030839, ...         Graduation   
122980  [-0.011204354465007782, 0.002425432438030839, ...            Camping   
122981  [-0.011204354465007782, 0.002425432438030839, ...             Beauty   
122982  [-0.011204354465007782, 0.002425432438030839, ...   Grocery Shopping   
122983  [-0.011204354465007782, 0.002425432438030839, ...            Wedding   

         

In [45]:
#Calculating tSNE clusters for each of the IAB labels.

results={}
for label in tqdm(iab_labels):
    mask = df_labels_embeddings['labels'].isin([label])
    df_tier1 = df_labels_embeddings[mask]
    if df_tier1.shape[0]//2<50:         #setting custom perplexity numbers
        perp=df_tier1.shape[0]//2       
    else:
        perp=50
    to_TSNE_list=df_tier1['embeddings'].to_list()
    to_TSNE_arr=np.array(to_TSNE_list)
    tsne=TSNE(n_components=2, random_state=42, perplexity=perp)
    X_embedded = tsne.fit_transform(to_TSNE_arr)
    results[label]=tsne.kl_divergence_


  0%|          | 0/160 [00:00<?, ?it/s]/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████| 160/160 [09:56<00:00,  3.73s/it]


In [46]:
count=0
problematic=[]
#new_problematic=[]
for k,v in results.items():
    if  v>1.5:   #Here v is an heuristic that I set after a bit of trial and error
        print(k,v)
        problematic.append(k)

In [47]:
# with open("data/problematic.pkl", 'wb') as f: #Save the problematic clusters to file
#     pkl.dump(problematic,f)

In [48]:
problematic

[]

#CODE TO PLOT THE TIER1 CLUSTERS

'''


BELOW IS ONLY OPTIONAL



'''

In [49]:
mask = df_labels_embeddings['labels'].isin(tier1)
df_tier1 = df_labels_embeddings[mask]

In [50]:
df_tier1.head

<bound method NDFrame.head of                                                embeddings  \
9       [-0.016414720565080643, -0.027218835428357124,...   
12      [-0.016414720565080643, -0.027218835428357124,...   
14      [-0.016414720565080643, -0.027218835428357124,...   
16      [-0.016414720565080643, -0.027218835428357124,...   
25      [-0.016414720565080643, -0.027218835428357124,...   
...                                                   ...   
122940  [-0.023140832781791687, -0.0161676537245512, -...   
122949  [-0.023140832781791687, -0.0161676537245512, -...   
122957  [-0.023140832781791687, -0.0161676537245512, -...   
122969  [-0.023140832781791687, -0.0161676537245512, -...   
122971  [-0.023140832781791687, -0.0161676537245512, -...   

                         labels          item  
9                        Travel         crane  
12                       Sports         crane  
14                       Movies         crane  
16                      Science         crane

In [51]:
df_tier1.shape

(24501, 3)

In [52]:
to_TSNE_list=df_tier1['embeddings'].to_list()

In [53]:
to_TSNE_arr=np.array(to_TSNE_list)

In [54]:
if df_tier1.shape[0]//2<50:
    perp=df_tier1.shape[0]//2
else:
    perp=50

In [55]:
tsne=TSNE(n_components=2, random_state=42, perplexity= perp) #df_tier1.shape[0]//2)
X_embedded = tsne.fit_transform(to_TSNE_arr)
tsne.kl_divergence_

-0.01773819327354431

In [56]:
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0:'x',1:'y'})
df_embeddings = df_embeddings.assign(label=df_tier1.labels.values)
df_embeddings = df_embeddings.assign(items=df_tier1.item.values)

In [57]:
df_embeddings.head()

,x,y,label,items
0,14.335808,-74.011803,Travel,crane
1,14.335808,-74.011803,Sports,crane
2,15.132290,-74.922592,Movies,crane
3,14.335808,-74.011803,Science,crane
4,14.335808,-74.011803,Automotive,crane


In [58]:
fig = px.scatter(
    df_embeddings, x='x', y='y',
    color='label', labels={'color': 'label'},
    hover_data=['items'], title = 'Tier1 Adwords HTML')
fig.show()

In [59]:
fig.write_html("Tier1.html")
